In [9]:
# %load_ext autoreload
# %autoreload 2

import sys
import os
import pandas as pd
from datetime import datetime, timezone, timedelta

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store


In [10]:
import hopsworks

# 🔐 Authenticate with Hopsworks
project = hopsworks.login()

# 🌟 Get the feature store
fs = project.get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=config.FEATURE_GROUP_MODEL_PREDICTION_VERSION
)

# 📦 Read and filter data
df = fg.read()
# Use fixed datetime since April data is not available
fixed_hour = df["pickup_hour"].dropna().max()
print(f"📅 Using most recent available hour for predictions: {fixed_hour}")
predictions = df[df["pickup_hour"] == fixed_hour]


2025-05-10 10:34:23,650 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-05-10 10:34:23,770 INFO: Initializing external client
2025-05-10 10:34:23,775 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 10:34:25,046 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.43s) 
📅 Using most recent available hour for predictions: 2024-01-01 00:00:00+00:00


In [11]:
top_locations = predictions.sort_values("predicted_rides", ascending=False)
top_locations[["pickup_location_id", "pickup_hour", "predicted_rides"]].head(10)


,pickup_location_id,pickup_hour,predicted_rides
2,6948.10,2024-01-01 00:00:00+00:00,9
1,5329.03,2024-01-01 00:00:00+00:00,4
0,6140.05,2024-01-01 00:00:00+00:00,3
